<a href="https://colab.research.google.com/github/AK18k/lora/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
        !git clone https://github.com/AK18k/lora

Cloning into 'lora'...
remote: Enumerating objects: 1360, done.
remote: Counting objects: 100% (1360/1360), done.
remote: Compressing objects: 100% (966/966), done.
remote: Total 1360 (delta 360), reused 1359 (delta 359), pack-reused 0
Receiving objects: 100% (1360/1360), 23.86 MiB | 19.09 MiB/s, done.
Resolving deltas: 100% (360/360), done.
